In [1]:
# Run this cell to import pyspark and to define start_spark() and stop_spark()

import findspark

findspark.init()

import getpass
import pandas
import pyspark
import random
import re

from IPython.display import display, HTML
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Functions used below

def username():
    """Get username with any domain information removed.
    """

    return re.sub('@.*', '', getpass.getuser())


def dict_to_html(d):
    """Convert a Python dictionary into a two column table for display.
    """

    html = []

    html.append(f'<table width="100%" style="width:100%; font-family: monospace;">')
    for k, v in d.items():
        html.append(f'<tr><td style="text-align:left;">{k}</td><td>{v}</td></tr>')
    html.append(f'</table>')

    return ''.join(html)


def show_as_html(df, n=20):
    """Leverage existing pandas jupyter integration to show a spark dataframe as html.
    
    Args:
        n (int): number of rows to show (default: 20)
    """

    display(df.limit(n).toPandas())

    
def display_spark():
    """Display the status of the active Spark session if one is currently running.
    """
    
    if 'spark' in globals() and 'sc' in globals():

        name = sc.getConf().get("spark.app.name")
        
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:green">active</span></b>, look for <code>{name}</code> under the running applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li>',
            f'<li><a href="{sc.uiWebUrl}" target="_blank">Spark Application UI</a></li>',
            f'</ul>',
            f'<p><b>Config</b></p>',
            dict_to_html(dict(sc.getConf().getAll())),
            f'<p><b>Notes</b></p>',
            f'<ul>',
            f'<li>The spark session <code>spark</code> and spark context <code>sc</code> global variables have been defined by <code>start_spark()</code>.</li>',
            f'<li>Please run <code>stop_spark()</code> before closing the notebook or restarting the kernel or kill <code>{name}</code> by hand using the link in the Spark UI.</li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))
        
    else:
        
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:red">stopped</span></b>, confirm that <code>{username() + " (jupyter)"}</code> is under the completed applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))


# Functions to start and stop spark

def start_spark(executor_instances=2, executor_cores=1, worker_memory=1, master_memory=1):
    """Start a new Spark session and define globals for SparkSession (spark) and SparkContext (sc).
    
    Args:
        executor_instances (int): number of executors (default: 2)
        executor_cores (int): number of cores per executor (default: 1)
        worker_memory (float): worker memory (default: 1)
        master_memory (float): master memory (default: 1)
    """

    global spark
    global sc

    user = username()
    
    cores = executor_instances * executor_cores
    partitions = cores * 4
    port = 4000 + random.randint(1, 999)

    spark = (
        SparkSession.builder
        .master("spark://masternode2:7077")
        .config("spark.driver.extraJavaOptions", f"-Dderby.system.home=/tmp/{user}/spark/")
        .config("spark.dynamicAllocation.enabled", "false")
        .config("spark.executor.instances", str(executor_instances))
        .config("spark.executor.cores", str(executor_cores))
        .config("spark.cores.max", str(cores))
        .config("spark.executor.memory", f"{worker_memory}g")
        .config("spark.driver.memory", f"{master_memory}g")
        .config("spark.driver.maxResultSize", "0")
        .config("spark.sql.shuffle.partitions", str(partitions))
        .config("spark.ui.port", str(port))
        .appName(user + " (jupyter)")
        .getOrCreate()
    )
    sc = SparkContext.getOrCreate()
    
    display_spark()

    
def stop_spark():
    """Stop the active Spark session and delete globals for SparkSession (spark) and SparkContext (sc).
    """

    global spark
    global sc

    if 'spark' in globals() and 'sc' in globals():

        spark.stop()

        del spark
        del sc

    display_spark()


# Make css changes to improve spark output readability

html = [
    '<style>',
    'pre { white-space: pre !important; }',
    'table.dataframe td { white-space: nowrap !important; }',
    'table.dataframe thead th:first-child, table.dataframe tbody th { display: none; }',
    '</style>',
]
display(HTML(''.join(html)))

In [2]:
# Run this cell to start a spark session in this notebook (don't actually use this many resources)

start_spark(executor_instances=4, executor_cores=2, worker_memory=4, master_memory=4)

spark.dynamicAllocation.enabled,false
spark.executor.instances,4
spark.driver.memory,4g
spark.executor.memory,4g
spark.app.id,app-20240523160731-0044
spark.master,spark://masternode2:7077
spark.executor.id,driver
spark.sql.warehouse.dir,file:/users/home/mji57/Assignment_2/spark-warehouse
spark.executor.cores,2
spark.driver.host,mathmadslinux2p.canterbury.ac.nz
spark.sql.shuffle.partitions,32


In [3]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Million Song Dataset Exploration") \
    .getOrCreate()

# Set the data path
data_path = "hdfs:///data/msd/main/summary/"

# Read the CSV data
main_summary_df = spark.read \
    .format("csv") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .option("compression", "gzip") \
    .load(data_path + "analysis.csv.gz")

# Show the schema and display the first few rows to understand the data structure
main_summary_df.printSchema()
main_summary_df.show(5)




root
 |-- analysis_sample_rate: integer (nullable = true)
 |-- audio_md5: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- end_of_fade_in: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- idx_bars_confidence: integer (nullable = true)
 |-- idx_bars_start: integer (nullable = true)
 |-- idx_beats_confidence: integer (nullable = true)
 |-- idx_beats_start: integer (nullable = true)
 |-- idx_sections_confidence: integer (nullable = true)
 |-- idx_sections_start: integer (nullable = true)
 |-- idx_segments_confidence: integer (nullable = true)
 |-- idx_segments_loudness_max: integer (nullable = true)
 |-- idx_segments_loudness_max_time: integer (nullable = true)
 |-- idx_segments_loudness_start: integer (nullable = true)
 |-- idx_segments_pitches: integer (nullable = true)
 |-- idx_segments_start: integer (nullable = true)
 |-- idx_segments_timbre: integer (nullable = true)
 |-- idx_tatums_confidence: integ

In [4]:
# Count total rows in the main summary dataset
total_rows_main_summary = main_summary_df.count()
print("Total rows in Main Summary Dataset:", total_rows_main_summary)

# Count unique track IDs in the main summary dataset
unique_tracks_main_summary = main_summary_df.select("track_id").distinct().count()
print("Unique tracks in Main Summary Dataset:", unique_tracks_main_summary)

# You would repeat similar counts for other datasets like the Taste Profile or any other relevant datasets.


Total rows in Main Summary Dataset: 1000000
Unique tracks in Main Summary Dataset: 1000000


In [5]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Read Compressed CSV Data") \
    .getOrCreate()

try:
    # Specify the correct path to the CSV file
    csv_file_path = "hdfs://masternode2:9000/data/msd/main/summary/analysis.csv.gz"

    # Read the compressed CSV data
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("compression", "gzip") \
        .load(csv_file_path)

    df.show(5)  # Display the first few rows to confirm successful load

except Exception as e:
    print("Failed to load data:", e)


+--------------------+--------------------+------------+---------+--------------+------+-------------------+--------------+--------------------+---------------+-----------------------+------------------+-----------------------+-------------------------+------------------------------+---------------------------+--------------------+------------------+-------------------+---------------------+----------------+---+--------------+--------+----+---------------+-----------------+-------+--------------+-------------------------+------------------+
|analysis_sample_rate|           audio_md5|danceability| duration|end_of_fade_in|energy|idx_bars_confidence|idx_bars_start|idx_beats_confidence|idx_beats_start|idx_sections_confidence|idx_sections_start|idx_segments_confidence|idx_segments_loudness_max|idx_segments_loudness_max_time|idx_segments_loudness_start|idx_segments_pitches|idx_segments_start|idx_segments_timbre|idx_tatums_confidence|idx_tatums_start|key|key_confidence|loudness|mode|mode_conf

In [14]:
# Assuming the mismatches are stored in a CSV file with fields 'wrong_track_id' and 'correct_track_id'
mismatches_path = "hdfs://masternode2:9000/data/msd/mismatches/mismatch_data.csv"
mismatches_df = spark.read.format("csv") \
    .option("header", "true") \
    .load(mismatches_path)

mismatches_df.show(5)  # Display some of the mismatches to understand the data


AnalysisException: Path does not exist: hdfs://masternode2:9000/data/msd/mismatches/mismatch_data.csv

In [11]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("Data Cleaning for MSD").getOrCreate()

# Paths to the datasets
main_data_path = "hdfs:///data/msd/main/summary/"
taste_profile_path = "hdfs:///data/msd/tasteprofile/"
mismatches_path = "hdfs:///data/msd/mismatches/"  # Hypothetical path

# Load the main dataset
main_df = spark.read.format("parquet").load(main_data_path)

# Load the Taste Profile dataset
taste_df = spark.read.format("parquet").load(taste_profile_path)

# Load mismatches (assuming a simple CSV format for demonstration)
mismatches_df = spark.read.format("csv").option("header", "true").load(mismatches_path)

# Assuming mismatches_df has columns 'wrong_track_id' and 'correct_track_id'
# Filtering out wrong entries
taste_df_cleaned = taste_df.join(mismatches_df, taste_df.track_id == mismatches_df.wrong_track_id, "left_anti")

# Optionally, if you want to correct mismatches instead of just removing them:
# Joining to get correct track_ids
corrected_taste_df = taste_df.join(mismatches_df, taste_df.track_id == mismatches_df.wrong_track_id, "left_outer") \
    .select(taste_df["*"], mismatches_df["correct_track_id"]) \
    .withColumn("track_id", coalesce(col("correct_track_id"), col("track_id"))) \
    .drop("correct_track_id", "wrong_track_id")

# Show results
corrected_taste_df.show(5)


Py4JJavaError: An error occurred while calling o472.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (192.168.40.182 executor 0): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:375)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:450)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:496)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:490)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$2(SchemaMergeUtils.scala:75)
	a

In [ ]:
# Close the Spark session
spark.stop()